In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import skfuzzy as fuzz
import math
import os

In [11]:
df = pd.read_parquet(r"data/pp_joind.parquet", engine="pyarrow")

In [12]:
na_counts = df.isna().sum()
na_counts_df = na_counts.reset_index()
na_counts_df.columns = ['Coluna', 'Qtd_NaNs']
print(na_counts_df)

df = df.dropna(axis=0)

                 Coluna  Qtd_NaNs
0                  neid         0
1           LOCALCELLID         0
2               dateday         0
3   cellreselpriority_x         0
4           qrxlevmin_x         0
..                  ...       ...
66        DLEARFCN_3200         0
67        DLEARFCN_3350         0
68        DLEARFCN_3375         0
69        DLEARFCN_9410         0
70        DLEARFCN_9510         0

[71 rows x 2 columns]


In [ ]:
tamanho_particao = math.ceil(len(df) * 0.1)

num_particao = 1
n_clusters = 3

for i in range(0, len(df), tamanho_particao):
    particao = df.iloc[i:i + tamanho_particao].copy()

    num_cols = particao.select_dtypes(include="number").columns
    X_scaled = StandardScaler().fit_transform(particao[num_cols])

    X_scaled_T = X_scaled.T

    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        X_scaled_T, c=n_clusters, m=2, error=0.005, maxiter=1000, init=None
    )

    labels = u.argmax(axis=0)
    particao["cluster_fuzzy"] = labels

    # Adiciona o grau de pertencimento de cada cluster como coluna
    for idx in range(n_clusters):
        particao[f"pertinencia_cluster_{idx}"] = u[idx]

    nome_arquivo = os.path.join("data", f"particao_{num_particao}.csv")
    particao.to_csv(nome_arquivo, index=False)
    print(f"Partição {num_particao} salva com {len(particao)} linhas")
    num_particao += 1

Partição 1 salva com 114305 linhas
Partição 2 salva com 114305 linhas
Partição 3 salva com 114305 linhas
Partição 4 salva com 114305 linhas
Partição 5 salva com 114305 linhas


KeyboardInterrupt: 

In [ ]:
# Lista das partições
particoes = [f"data/particao_{i}.csv" for i in range(1, 11)]

# Criar pasta results se não existir
os.makedirs("results", exist_ok=True)

for caminho in particoes:
    df = pd.read_csv(caminho, engine="pyarrow")

    if not all(col in df.columns for col in ['cluster_fuzzy']):
        print(f"ERROR: {caminho} não possui pca1, pca2 e cluster_fuzzy. Pulando.")
        continue

    plt.figure(figsize=(8, 6))
    plt.scatter(df['TIM_THROU_USER_PDCP_DL (Kbps)'], df['TIM_PRB_UTIL_MEAN_DL (%)'], c=df['cluster_fuzzy'], cmap='viridis', s=10)
    plt.title(f'Fuzzy C Means Throughput vs PRB util - {os.path.basename(caminho)}')
    plt.xlabel('Thoruput DL (Kbps)')
    plt.ylabel('PRB Util Mean DL (%)')
    

    # Salvar gráfico em results
    nome_saida = os.path.join("results", f"{os.path.basename(caminho).replace('.csv', '')}_fuzzy.png")
    plt.savefig(nome_saida, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"Gráfico salvo em {nome_saida}")

Gráfico salvo em results\particao_1_fuzzy.png
Gráfico salvo em results\particao_2_fuzzy.png
Gráfico salvo em results\particao_3_fuzzy.png
Gráfico salvo em results\particao_4_fuzzy.png
Gráfico salvo em results\particao_5_fuzzy.png
Gráfico salvo em results\particao_6_fuzzy.png
Gráfico salvo em results\particao_7_fuzzy.png
Gráfico salvo em results\particao_8_fuzzy.png
Gráfico salvo em results\particao_9_fuzzy.png
Gráfico salvo em results\particao_10_fuzzy.png
